In [ ]:
import cobra
from cobra.flux_analysis import flux_variability_analysis
from cobra import Model, Reaction, Metabolite

In [ ]:
DraftTmurisV3 = cobra.io.read_sbml_model("Draft_Tmuris_V3.xml")


In [ ]:
ZeroFlux1 = []
i = 0
while i < len(DraftTmurisV3.reactions):
    FVA = flux_variability_analysis(DraftTmurisV3,DraftTmurisV3.reactions[i])
    if float(FVA.maximum) == 0.0 and float(FVA.minimum) == 0.0:
        ZeroFlux1.append(DraftTmurisV3.reactions[i].id)
        
    i += 1

In [ ]:
i = 0
while i < len(DraftTmurisV3.reactions):
    for RxnProducts in DraftTmurisV3.reactions[i].products:
        if RxnProducts.compartment == "c":
            if len(RxnProducts.reactions) == 1:
                SinkMet = RxnProducts
                print('The number of blocked reactions is', len(ZeroFlux1) ,'before ', SinkMet, 'is temporarily added to the draft model.\n')
                with DraftTmurisV3 as model:
                    RxnID = "sink"+str(RxnProducts)
                    reaction = Reaction(RxnID)
                    reaction.name = "sink"
                    reaction.lower_bound = -999999.0
                    reaction.upper_bound = 999999.0
                    reaction.add_metabolites({SinkMet: 1.0})
                    model.add_reactions([reaction])
                    ZeroFlux2 = []
                    t = 0
                    while t < len(model.reactions):
                        FVA = flux_variability_analysis(model,model.reactions[t])
                        if float(FVA.maximum) == 0.0 and float(FVA.minimum) == 0.0:
                            ZeroFlux2.append(model.reactions[t].id)
                        t += 1
                    
                    print(RxnProducts.id,'is added!','The number of blocked reaction:\t', len(ZeroFlux2),'\n')
    i += 1